In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog, minimize
from ticker_download_predict_upload import DownloadPredictUpload

In [ ]:
dpu = DownloadPredictUpload()

### Set the tickers in the portfolio

In [ ]:
tickers = ["I:SPX", "QQQ", "VXUS", "GLD"]

### Get the past year of ticker close prices

In [ ]:
long_df_filename = os.path.join("input", f"Year of Tickers {dpu.get_today_date()}.csv")
if os.path.exists(long_df_filename):
    long_df = pd.read_csv(long_df_filename)
    long_df["datetime"] = pd.to_datetime(long_df["datetime"])
    long_df["datetime"] = long_df["datetime"].apply(
        lambda x: pd.Timestamp(x).replace(hour=23, minute=59, second=59)
    )
    long_df.set_index("datetime", inplace=True)
    long_df.sort_index(inplace=True)
else:
    date_from = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 253)
    date_to = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 1).replace(
        hour=23, minute=59, second=59
    )
    print(date_from, date_to)
    long_df = dpu.get_tickers(tickers, date_from=date_from, date_to=date_to)
    long_df.to_csv(long_df_filename, index=True)
long_df

### Pivot the close prices for better analysis

In [ ]:
wide_df = dpu.pivot_ticker_close_wide(long_df)
wide_df

### Are there any missing values?

In [ ]:
wide_df.isna().sum().sum()

### Calculate % change and covert to a percentage

In [ ]:
returns_df = wide_df.pct_change()
returns_df = returns_df.iloc[1:] * 100
returns_df

### Calculate mean returns

In [ ]:
mean_returns = returns_df.mean()
mean_returns

### Calculate covariance matrix

In [ ]:
cov = returns_df.cov()
cov_np = cov.to_numpy()
cov

### Simulate 10,000 portfolios by generating random weights

In [ ]:
n_portfolios = 10_000
d = len(mean_returns)
simulated_returns = np.zeros(n_portfolios)
simulated_risks = np.zeros(n_portfolios)
random_weights = []
rand_range = 1.0

for i in range(n_portfolios):
    w = np.random.random(d) * rand_range - rand_range / 2  # Allows short-selling
    w[-1] = 1 - w[:-1].sum()
    np.random.shuffle(w)
    random_weights.append(w)
    simulated_return = mean_returns.dot(w)
    simulated_risk = np.sqrt(w.dot(cov_np).dot(w))
    simulated_returns[i] = simulated_return
    simulated_risks[i] = simulated_risk

### Optimize minimum and maximum return portfolios

To get bounds for calculating efficient frontier

In [ ]:
D = len(tickers)  # Number of assets in portfolio
A_eq = np.ones((1, D))
b_eq = np.ones(1)
weight_bounds = [(-0.5, None)] * D
weight_bounds

In [ ]:
minimum_return_result = linprog(
    mean_returns, A_eq=A_eq, b_eq=b_eq, bounds=weight_bounds
)
minimum_return_result

In [ ]:
maximum_return_result = linprog(
    mean_returns, A_eq=A_eq, b_eq=b_eq, bounds=weight_bounds
)
maximum_return_result

In [ ]:
min_return = minimum_return_result.fun
max_return = -maximum_return_result.fun
min_return, max_return

### Calculate efficient frontier line

a.k.a. mean-variance optimal portfolios

In [ ]:
num_portfolios = 100
target_returns = np.linspace(min_return, max_return, num_portfolios)
target_returns

In [ ]:
def get_portfolio_variance(weights):
    return weights.dot(cov_np).dot(weights)

In [ ]:
def target_returns_constraint(weights, target_return):
    return weights.dot(mean_returns) - target_return

In [ ]:
def portfolio_weights_constraint(weights):
    return weights.sum() - 1

In [ ]:
constraints = [
    {"type": "eq", "fun": target_returns_constraint, "args": [target_returns[0]]},
    {"type": "eq", "fun": portfolio_weights_constraint},
]

In [ ]:
optimized_risks = []
for target_return in target_returns:
    constraints[0]["args"] = [target_return]
    result = minimize(
        fun=get_portfolio_variance,
        x0=np.ones(D) / D,
        method="SLSQP",
        bounds=weight_bounds,
        constraints=constraints,
    )
    if result.status == 0:
        optimized_risks.append(np.sqrt(result.fun))
    else:
        print("Optimization error!", result)

### Calculate Sharpe Ratio

In [ ]:
risk_free_rate_pct = 4.2 / 252  # 252 trading days per year because of daily data
risk_free_rate_pct